<a href="https://colab.research.google.com/github/keirace/artificial-guacamole/blob/master/YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Connect to google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/Shareddrives/ /Shareddrives
!nvidia-smi

Mounted at /content/gdrive
Wed May 12 13:07:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                             

# **1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15035, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 15035 (delta 11), reused 18 (delta 8), pack-reused 15002
Receiving objects: 100% (15035/15035), 13.44 MiB | 12.03 MiB/s, done.
Resolving deltas: 100% (10210/10210), done.


# **2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

# **3) Prepare Data**

In [ ]:
!echo -e "Defective\nNormal" > data/obj.names
!echo -e 'classes= 2\ntrain = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /Shareddrives/caps/weights\nvalid = data/train.txt' > data/obj.data
!mkdir data/obj

In [ ]:
from os.path import join 
train_path = '/content/gdrive/Shareddrives/caps/data/train'
test_path = '/content/gdrive/Shareddrives/caps/data/test'

In [ ]:
import glob
import os
import re
def generate(path):
  images_list = [f for f in glob.glob(join(path, "*.jpg")) if os.path.exists(f.split('.jpg')[0] + '.txt')]
  print("number of images: "  , len(images_list))

  #Create training.txt file
  file = open("data/train.txt", "w") if 'train' in path else open("data/test.txt", "w")
  file.write("\n".join(images_list)) 
  file.close() 

generate(train_path)
generate(test_path)

number of images:  213
number of images:  44


In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# **YOLOv4**

## **4) Configure Darknet network for training YOLO V4**

In [ ]:
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg
!sed -i 's/subdivisions=8/subdivisions=32/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolov4_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov4_training.cfg
!sed -i 's/width=608/width=512/' cfg/yolov4_training.cfg
!sed -i 's/height=608/height=512/' cfg/yolov4_training.cfg
!sed -i '968 s@classes=80@classes=2@' cfg/yolov4_training.cfg
!sed -i '1056 s@classes=80@classes=2@' cfg/yolov4_training.cfg
!sed -i '1144 s@classes=80@classes=2@' cfg/yolov4_training.cfg
!sed -i '961 s@filters=255@filters=21@' cfg/yolov4_training.cfg
!sed -i '1049 s@filters=255@filters=21@' cfg/yolov4_training.cfg
!sed -i '1137 s@filters=255@filters=21@' cfg/yolov4_training.cfg

## **5) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4_training.cfg /Shareddrives/caps/weights/yolov4.conv.137 -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 1041436, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 1, class_loss = 0.000010, iou_loss = 0.000000, total_loss = 0.000010 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.872892), count: 6, class_loss = 0.000894, iou_loss = 1.317697, total_loss = 1.318590 
 total_bbox = 1041442, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.250946), count: 1, class_loss = 0.696145, iou_loss = 2.037690, total_loss = 2.733835 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.904009), count: 12, class_loss = 0.013604, iou_loss = 13.219818, total_loss = 13.23342

### **5-1) Continue training**

In [ ]:
# Continue the training
# !./darknet detector train data/obj.data cfg/yolov4_training.cfg /Shareddrives/caps/weights/yolov4_training_last.weights -dont_show -map

## **6) mAP**

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4_training.cfg /Shareddrives/caps/weights/yolov4_training_4000.weights

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/'  yolov4_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1/'  yolov4_training.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4_training.cfg /Shareddrives/caps/weights/yolov4_training_4000.weights /Shareddrives/caps/data/test/5CB08291-6F01-4158-BEB7-E5672C37852E.jpg -thresh 0.3
imShow('predictions.jpg')

## **7) Save results as .txt file**
multiple images

In [ ]:
!./darknet detector test data/obj.data /Shareddrives/caps/weights/yolov4_training_4000.weights -dont_show -ext_output < /Shareddrives/caps/data/images.txt > /Shareddrives/caps/results/yolov4_result.txt

# **YOLOv4x-mish**

## **4) Configure Darknet network for training YOLO V4x-mish**

In [ ]:
!cp cfg/yolov4x-mish.cfg cfg/yolov4x-mish_training.cfg
!sed -i 's/subdivisions=8/subdivisions=32/' cfg/yolov4x-mish_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolov4x-mish_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov4x-mish_training.cfg
!sed -i 's/width=640/width=512/' cfg/yolov4x-mish_training.cfg
!sed -i 's/height=640/height=512/' cfg/yolov4x-mish_training.cfg
!sed -i 's/#random=1/random=1/' cfg/yolov4x-mish_training.cfg
!sed -i '1159 s@classes=80@classes=2@' cfg/yolov4x-mish_training.cfg
!sed -i '1289 s@classes=80@classes=2@' cfg/yolov4x-mish_training.cfg
!sed -i '1419 s@classes=80@classes=2@' cfg/yolov4x-mish_training.cfg
!sed -i '1152 s@filters=255@filters=21@' cfg/yolov4x-mish_training.cfg
!sed -i '1282 s@filters=255@filters=21@' cfg/yolov4x-mish_training.cfg
!sed -i '1412 s@filters=255@filters=21@' cfg/yolov4x-mish_training.cfg

## **5) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4x-mish_training.cfg /Shareddrives/caps/weights/yolov4x-mish.conv.166 -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.05, obj: 4.00, cls: 0.50) Region 168 Avg (IOU: 0.000000), count: 1, class_loss = 1.464767, iou_loss = -1.098575, total_loss = 0.366192 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 1.00, cls: 0.50) Region 185 Avg (IOU: 0.692781), count: 2, class_loss = 2.881848, iou_loss = 0.057716, total_loss = 2.939564 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 0.40, cls: 0.50) Region 202 Avg (IOU: 0.651370), count: 6, class_loss = 0.504900, iou_loss = 1.000170, total_loss = 1.505070 
 total_bbox = 496516, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 4.00, cls: 0.50) Region 168 Avg (IOU: 0.000000), count: 1, class_loss = 1.598491, iou_loss = -1.198868, total_loss = 0.399623 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 1.00, cls: 0.50) Region 185 Avg (IOU: 0.800714), count: 2, class_loss = 3.831775, iou_loss = 0.138189, total_loss = 3.969964 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 0.40, cl

### **5-1) Continue training**

In [ ]:
# Continue the training
!./darknet detector train data/obj.data cfg/yolov4x-mish_training.cfg /Shareddrives/caps/weights/yolov4x-mish_training_last.weights -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.05, obj: 4.00, cls: 0.50) Region 168 Avg (IOU: 0.736875), count: 5, class_loss = 40.958088, iou_loss = -28.465366, total_loss = 12.492723 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 1.00, cls: 0.50) Region 185 Avg (IOU: 0.820955), count: 17, class_loss = 4.986779, iou_loss = 5.304217, total_loss = 10.290996 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 0.40, cls: 0.50) Region 202 Avg (IOU: 0.672538), count: 5, class_loss = 0.049563, iou_loss = 0.751451, total_loss = 0.801014 
 total_bbox = 145919, rewritten_bbox = 0.000685 % 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 4.00, cls: 0.50) Region 168 Avg (IOU: 0.494967), count: 5, class_loss = 97.374123, iou_loss = -68.278229, total_loss = 29.095894 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 1.00, cls: 0.50) Region 185 Avg (IOU: 0.799350), count: 7, class_loss = 3.245376, iou_loss = 1.218777, total_loss = 4.464153 
v3 (iou loss, Normalizer: (iou: 0.05, obj: 

## **6) mAP**

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4x-mish_training.cfg /Shareddrives/caps/weights/yolov4x-mish_4000.weights

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/'  yolov4x-mish_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1/'  yolov4x-mish_training.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4x-mish_training.cfg /Shareddrives/caps/weights/yolov4x-mish_training_4000.weights /Shareddrives/caps/data/test/5CB08291-6F01-4158-BEB7-E5672C37852E.jpg -thresh 0.3
imShow('predictions.jpg')

## **7) Save results as .txt file**
multiple images

In [ ]:
!./darknet detector test data/obj.data /Shareddrives/caps/weights/yolov4x-mish_training_4000.weights -dont_show -ext_output < /Shareddrives/caps/data/images.txt > /Shareddrives/caps/results/yolov4_result.txt

# **YOLOv4-csp**

## **4) Configure Darknet network for training YOLO V4-csp**

In [ ]:
!cp cfg/yolov4-csp.cfg cfg/yolov4-csp_training.cfg
!sed -i 's/subdivisions=8/subdivisions=16/' cfg/yolov4-csp_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolov4-csp_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov4-csp_training.cfg
!sed -i 's/#random=1/random=1/' cfg/yolov4-csp_training.cfg
!sed -i '1034 s@classes=80@classes=2@' cfg/yolov4-csp_training.cfg
!sed -i '1148 s@classes=80@classes=2@' cfg/yolov4-csp_training.cfg
!sed -i '1262 s@classes=80@classes=2@' cfg/yolov4-csp_training.cfg
!sed -i '1027 s@filters=255@filters=21@' cfg/yolov4-csp_training.cfg
!sed -i '1141 s@filters=255@filters=21@' cfg/yolov4-csp_training.cfg
!sed -i '1255 s@filters=255@filters=21@' cfg/yolov4-csp_training.cfg

## **5) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4-csp_training.cfg /Shareddrives/caps/weights/yolov4-csp.conv.142 -dont_show -map

### **5-1) Continue training**

In [ ]:
# Continue the training
# !./darknet detector train data/obj.data cfg/yolov4-csp_training.cfg /Shareddrives/caps/weights/yolov4-csp_training_last.weights -dont_show -map

## **6) mAP**

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-csp_training.cfg /Shareddrives/caps/weights/yolov4-csp_4000.weights

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/'  yolov4-csp_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1/'  yolov4-csp_training.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-csp_training.cfg /Shareddrives/caps/weights/yolov4-csp_training_4000.weights /Shareddrives/caps/data/test/5CB08291-6F01-4158-BEB7-E5672C37852E.jpg -thresh 0.3
imShow('predictions.jpg')

## **7) Save results as .txt file**
multiple images

In [ ]:
!./darknet detector test data/obj.data /Shareddrives/caps/weights/yolov4-csp_training_4000.weights -dont_show -ext_output < /Shareddrives/caps/data/images.txt > /Shareddrives/caps/results/yolov4_result.txt

# **YOLOv4 tiny**

## **4) Configure Darknet network for training YOLO V4 tiny**

In [ ]:
!cp cfg/yolov4-tiny.cfg cfg/yolov4-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny_training.cfg
!sed -i 's/max_batches = 2000200/max_batches = 6000/' cfg/yolov4-tiny_training.cfg
!sed -i 's/steps=1600000,1800000/steps=4800,5400/' cfg/yolov4-tiny_training.cfg
!sed -i 's/width=416/width=512/' cfg/yolov4-tiny_training.cfg
!sed -i 's/height=416/height=512/' cfg/yolov4-tiny_training.cfg
!sed -i 's/random=0/random=1/' cfg/yolov4-tiny_training.cfg
!sed -i '229 s@classes=80@classes=2@' cfg/yolov4-tiny_training.cfg
!sed -i '280 s@classes=80@classes=2@' cfg/yolov4-tiny_training.cfg
!sed -i '221 s@filters=255@filters=21@' cfg/yolov4-tiny_training.cfg
!sed -i '274 s@filters=255@filters=21@' cfg/yolov4-tiny_training.cfg

## **5) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4-tiny_training.cfg /Shareddrives/caps/weights/yolov4-tiny.conv.29 -dont_show -map

### **5-1) Continue training**

In [ ]:
# Continue the training
# !./darknet detector train data/obj.data cfg/yolov4-csp_training.cfg /Shareddrives/caps/weights/yolov4-tiny_training_last.weights -dont_show -map

## **6) mAP**

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny_training.cfg /Shareddrives/caps/weights/yolov4-tiny_4000.weights

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/'  yolov4-tiny_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1/'  yolov4-tiny_training.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-tiny_training.cfg /Shareddrives/caps/weights/yolov4-tiny_training_4000.weights /Shareddrives/caps/data/test/5CB08291-6F01-4158-BEB7-E5672C37852E.jpg -thresh 0.3
imShow('predictions.jpg')

## **7) Save results as .txt file**
multiple images

In [ ]:
!./darknet detector test data/obj.data /Shareddrives/caps/weights/yolov4-tiny_training_4000.weights -dont_show -ext_output < /Shareddrives/caps/data/images.txt > /Shareddrives/caps/results/yolov4_result.txt